Shared Bank Account Simulation

In [ ]:
%%writefile l4_t1.c

#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <time.h>

#define INITIAL_BALANCE 1000
#define TRANSACTIONS_PER_THREAD 5
#define MAX_AMOUNT 100

// Shared bank account balance
int account_balance = INITIAL_BALANCE;

// Mutex for synchronizing access to the account balance
pthread_mutex_t mutex;

void* perform_transactions(void* thread_id) {
    int id = *((int*)thread_id);
    free(thread_id); // Free the allocated memory for thread_id

    for (int i = 0; i < TRANSACTIONS_PER_THREAD; i++) {
        // Randomly choose to deposit or withdraw
        int amount = rand() % MAX_AMOUNT + 1; // Amount between 1 and 100
        int action = rand() % 2; // 0 for deposit, 1 for withdraw

        pthread_mutex_lock(&mutex); // Lock the mutex

        if (action == 0) { // Deposit
            account_balance += amount;
            printf("Thread %d: Deposited %d. New balance: %d\n", id, amount, account_balance);
        } else { // Withdraw
            if (account_balance >= amount) {
                account_balance -= amount;
                printf("Thread %d: Withdrew %d. New balance: %d\n", id, amount, account_balance);
            } else {
                printf("Thread %d: Attempted to withdraw %d but insufficient funds. Balance remains: %d\n", id, amount, account_balance);
            }
        }

        pthread_mutex_unlock(&mutex); // Unlock the mutex
    }

    return NULL;
}

int main() {
    srand(time(NULL)); // Seed the random number generator

    pthread_t threads[4];
    pthread_mutex_init(&mutex, NULL); // Initialize the mutex

    // Create and start threads
    for (int i = 0; i < 4; i++) {
        int* thread_id = malloc(sizeof(int)); // Allocate memory for thread ID
        *thread_id = i + 1; // Assign thread ID (1 to 4)
        pthread_create(&threads[i], NULL, perform_transactions, thread_id);
    }

    // Wait for all threads to finish
    for (int i = 0; i < 4; i++) {
        pthread_join(threads[i], NULL);
    }

    // Print final account balance
    printf("Final account balance: %d\n", account_balance);

    pthread_mutex_destroy(&mutex); // Destroy the mutex

    return 0;
}


Writing l4_t1.c


In [ ]:
!gcc l4_t1.c

In [ ]:
!./a.out

Thread 2: Withdrew 4. New balance: 996
Thread 2: Deposited 11. New balance: 1007
Thread 2: Deposited 15. New balance: 1022
Thread 2: Deposited 43. New balance: 1065
Thread 2: Deposited 74. New balance: 1139
Thread 1: Deposited 20. New balance: 1159
Thread 1: Withdrew 60. New balance: 1099
Thread 1: Withdrew 3. New balance: 1096
Thread 1: Deposited 78. New balance: 1174
Thread 1: Withdrew 32. New balance: 1142
Thread 3: Withdrew 93. New balance: 1049
Thread 3: Deposited 37. New balance: 1086
Thread 3: Deposited 37. New balance: 1123
Thread 3: Deposited 8. New balance: 1131
Thread 3: Withdrew 54. New balance: 1077
Thread 4: Deposited 64. New balance: 1141
Thread 4: Deposited 75. New balance: 1216
Thread 4: Deposited 51. New balance: 1267
Thread 4: Deposited 23. New balance: 1290
Thread 4: Deposited 94. New balance: 1384
Final account balance: 1384


Racing Game Simulation

In [ ]:
%%writefile l4_t2.c

#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <unistd.h>

#define NUM_CARS 4
#define NUM_CHECKPOINTS 3

// Barrier structure
typedef struct {
    pthread_mutex_t mutex;
    pthread_cond_t cond;
    int count;
    int total;
} Barrier;

// Global barrier variable
Barrier barrier;

// Initialize the barrier
void barrier_init(Barrier* barrier, int total) {
    pthread_mutex_init(&barrier->mutex, NULL);
    pthread_cond_init(&barrier->cond, NULL);
    barrier->count = 0;
    barrier->total = total;
}

// Wait at the barrier
void barrier_wait(Barrier* barrier) {
    pthread_mutex_lock(&barrier->mutex);

    barrier->count++;

    if (barrier->count == barrier->total) {
        // Reset the count and signal all waiting threads
        barrier->count = 0;
        pthread_cond_broadcast(&barrier->cond);
    } else {
        // Wait until all cars reach the checkpoint
        pthread_cond_wait(&barrier->cond, &barrier->mutex);
    }

    pthread_mutex_unlock(&barrier->mutex);
}

// Car thread function
void* car_race(void* car_id) {
    int id = *((int*)car_id);
    free(car_id); // Free allocated memory for car_id

    for (int checkpoint = 1; checkpoint <= NUM_CHECKPOINTS; checkpoint++) {
        // Simulate reaching the checkpoint
        printf("Car %d has reached checkpoint %d\n", id, checkpoint);

        // Wait at the barrier for other cars
        barrier_wait(&barrier);

        // Simulate some time delay before moving to the next checkpoint
        sleep(1);
    }

    return NULL;
}

int main() {
    pthread_t cars[NUM_CARS];

    // Initialize the barrier for NUM_CARS
    barrier_init(&barrier, NUM_CARS);

    // Create and start car threads
    for (int i = 0; i < NUM_CARS; i++) {
        int* car_id = malloc(sizeof(int)); // Allocate memory for car ID
        *car_id = i + 1; // Assign car ID (1 to 4)
        pthread_create(&cars[i], NULL, car_race, car_id);
    }

    // Wait for all car threads to finish
    for (int i = 0; i < NUM_CARS; i++) {
        pthread_join(cars[i], NULL);
    }

    // Clean up resources
    pthread_mutex_destroy(&barrier.mutex);
    pthread_cond_destroy(&barrier.cond);

    return 0;
}


Overwriting l4_t2.c


In [ ]:
!gcc l4_t2.c

In [ ]:
!./a.out

Car 3 has reached checkpoint 1
Car 2 has reached checkpoint 1
Car 4 has reached checkpoint 1
Car 1 has reached checkpoint 1
Car 4 has reached checkpoint 2
Car 2 has reached checkpoint 2
Car 1 has reached checkpoint 2
Car 3 has reached checkpoint 2
Car 4 has reached checkpoint 3
Car 2 has reached checkpoint 3
Car 1 has reached checkpoint 3
Car 3 has reached checkpoint 3


Matrix Multiplication

In [ ]:
%%writefile l4_t3.c

#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <time.h>

#define SIZE 4 // Size of the matrices
#define NUM_THREADS 4 // Number of threads

// Matrices
int A[SIZE][SIZE];
int B[SIZE][SIZE];
int C[SIZE][SIZE];

// Mutex for protecting access to the result matrix C
pthread_mutex_t mutex;

// Barrier structure
typedef struct {
    pthread_mutex_t mutex;
    pthread_cond_t cond;
    int count;
    int total;
} Barrier;

Barrier barrier;

// Initialize the barrier
void barrier_init(Barrier* barrier, int total) {
    pthread_mutex_init(&barrier->mutex, NULL);
    pthread_cond_init(&barrier->cond, NULL);
    barrier->count = 0;
    barrier->total = total;
}

// Wait at the barrier
void barrier_wait(Barrier* barrier) {
    pthread_mutex_lock(&barrier->mutex);

    barrier->count++;

    if (barrier->count == barrier->total) {
        // Reset the count and signal all waiting threads
        barrier->count = 0;
        pthread_cond_broadcast(&barrier->cond);
    } else {
        // Wait until all threads reach the barrier
        pthread_cond_wait(&barrier->cond, &barrier->mutex);
    }

    pthread_mutex_unlock(&barrier->mutex);
}

// Function to multiply a row of A with B and store in C
void* multiply_row(void* arg) {
    int row = *((int*)arg);

    for (int j = 0; j < SIZE; j++) {
        C[row][j] = 0; // Initialize C[row][j]
        for (int k = 0; k < SIZE; k++) {
            C[row][j] += A[row][k] * B[k][j];
        }
    }

    // Synchronize at the end of row calculation
    barrier_wait(&barrier);

    return NULL;
}

// Function to print a matrix
void print_matrix(int matrix[SIZE][SIZE]) {
    for (int i = 0; i < SIZE; i++) {
        for (int j = 0; j < SIZE; j++) {
            printf("%d ", matrix[i][j]);
        }
        printf("\n");
    }
}

int main() {
    srand(time(NULL)); // Seed for random number generation

    // Generate random matrices A and B
    for (int i = 0; i < SIZE; i++) {
        for (int j = 0; j < SIZE; j++) {
            A[i][j] = rand() % 10; // Random values between 0 and 9
            B[i][j] = rand() % 10; // Random values between 0 and 9
        }
    }

    // Print matrices A and B
    printf("Matrix A:\n");
    print_matrix(A);

    printf("Matrix B:\n");
    print_matrix(B);

    // Initialize the barrier for NUM_THREADS
    barrier_init(&barrier, NUM_THREADS);

    pthread_t threads[NUM_THREADS];

    // Create threads to multiply rows
    for (int i = 0; i < NUM_THREADS; i++) {
        int* row_id = malloc(sizeof(int)); // Allocate memory for row index
        *row_id = i; // Assign row index (0 to 3)
        pthread_create(&threads[i], NULL, multiply_row, row_id);
    }

    // Wait for all threads to finish
    for (int i = 0; i < NUM_THREADS; i++) {
        pthread_join(threads[i], NULL);
    }

    // Print the result matrix C
    printf("Result Matrix C:\n");
    print_matrix(C);

    // Clean up resources
    pthread_mutex_destroy(&mutex);
    pthread_mutex_destroy(&barrier.mutex);
    pthread_cond_destroy(&barrier.cond);

    return 0;
}


Writing l4_t3.c


In [ ]:
!gcc l4_t3.c

In [ ]:
!./a.out

Matrix A:
5 5 7 1 
4 6 5 3 
2 7 5 1 
6 8 4 9 
Matrix B:
1 4 6 2 
1 1 4 8 
1 3 1 7 
0 9 7 1 
Result Matrix C:
17 55 64 100 
15 64 74 94 
14 39 52 96 
18 125 135 113 


Producer-Consumer Problem

In [ ]:
%%writefile l5t1.c

#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <unistd.h>

#define BUFFER_SIZE 1  // Size of the buffer (1 for simplicity)

int buffer;  // Shared buffer
int produced = 0;  // Flag to indicate if an item is produced
pthread_mutex_t mutex;  // Mutex for synchronizing access to the buffer
pthread_cond_t cond_var;  // Condition variable for signaling

// Producer function
void* producer(void* arg) {
    while (1) {
        int num = rand() % 100;  // Produce a random number
        pthread_mutex_lock(&mutex);  // Lock the mutex
        buffer = num;  // Place the produced item in the buffer
        produced = 1;  // Set flag to indicate an item is produced
        printf("Producer: Produced data: %d\n", num);
        pthread_cond_signal(&cond_var);  // Signal the consumer
        pthread_mutex_unlock(&mutex);  // Unlock the mutex
        sleep(rand() % 2 + 1);  // Sleep for a random time (1-2 seconds)
    }
}

// Consumer function
void* consumer(void* arg) {
    while (1) {
        pthread_mutex_lock(&mutex);  // Lock the mutex
        while (produced == 0) {  // Wait until there is something to consume
            printf("Consumer: Waiting for the producer to produce...\n");
            pthread_cond_wait(&cond_var, &mutex);  // Wait for signal from producer
        }
        printf("Consumer: Consumed data: %d\n", buffer);
        produced = 0;  // Reset flag after consuming
        pthread_mutex_unlock(&mutex);  // Unlock the mutex
        sleep(rand() % 2 + 1);  // Sleep for a random time (1-2 seconds)
    }
}

int main() {
    pthread_t prod_thread, cons_thread;

    // Initialize mutex and condition variable
    pthread_mutex_init(&mutex, NULL);
    pthread_cond_init(&cond_var, NULL);

    // Create producer and consumer threads
    pthread_create(&prod_thread, NULL, producer, NULL);
    pthread_create(&cons_thread, NULL, consumer, NULL);

    // Join threads (not necessary in this infinite loop example)
    pthread_join(prod_thread, NULL);
    pthread_join(cons_thread, NULL);

    // Clean up (not reached in this example)
    pthread_mutex_destroy(&mutex);
    pthread_cond_destroy(&cond_var);

    return 0;
}


Overwriting l5t1.c


In [ ]:
!gcc l5t1.c

In [ ]:
!./a.out

Consumer: Waiting for the producer to produce...
Producer: Produced data: 83
Consumer: Consumed data: 83
Producer: Produced data: 15
Consumer: Consumed data: 15
Producer: Produced data: 86
Consumer: Consumed data: 86
Producer: Produced data: 49
Producer: Produced data: 27
Consumer: Consumed data: 27
Producer: Produced data: 63
Producer: Produced data: 40
Consumer: Consumed data: 40
Producer: Produced data: 36
Consumer: Consumed data: 36
Consumer: Waiting for the producer to produce...
Producer: Produced data: 67
Consumer: Consumed data: 67
Consumer: Waiting for the producer to produce...
Producer: Produced data: 30
Consumer: Consumed data: 30
Producer: Produced data: 67
Consumer: Consumed data: 67
Producer: Produced data: 2
Consumer: Consumed data: 2
Producer: Produced data: 69
Consumer: Consumed data: 69
Producer: Produced data: 56
Consumer: Consumed data: 56
Producer: Produced data: 29
Consumer: Consumed data: 29
Producer: Produced data: 19
Consumer: Consumed data: 19
Producer: Produ

Traffic Light Simulation

In [ ]:
%%writefile l5t2.c

#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <unistd.h>

// Mutex and condition variables
pthread_mutex_t mutex = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t road_condition = PTHREAD_COND_INITIALIZER;

// Traffic light state
enum RoadState {
    ROAD_A_GREEN,
    ROAD_B_GREEN
};
volatile enum RoadState current_state = ROAD_A_GREEN;
volatile int stop_simulation = 0;

// Road A thread function
void* road_a_thread(void* arg) {
    while (!stop_simulation) {
        pthread_mutex_lock(&mutex);

        // Wait if not green for Road A
        while (current_state != ROAD_A_GREEN) {
            printf("Road A: Red light. Waiting...\n");
            pthread_cond_wait(&road_condition, &mutex);
        }

        // Road A is green
        printf("Road A: Green light! Cars passing...\n");
        sleep(2); // Simulate cars passing

        pthread_mutex_unlock(&mutex);
        sleep(1);
    }
    return NULL;
}

// Road B thread function
void* road_b_thread(void* arg) {
    while (!stop_simulation) {
        pthread_mutex_lock(&mutex);

        // Wait if not green for Road B
        while (current_state != ROAD_B_GREEN) {
            printf("Road B: Red light. Waiting...\n");
            pthread_cond_wait(&road_condition, &mutex);
        }

        // Road B is green
        printf("Road B: Green light! Cars passing...\n");
        sleep(2); // Simulate cars passing

        pthread_mutex_unlock(&mutex);
        sleep(1);
    }
    return NULL;
}

// Traffic light controller thread function
void* traffic_light_controller(void* arg) {
    int cycle_count = 0;

    while (cycle_count < 4) {  // Simulate 4 cycles
        // Road A green
        pthread_mutex_lock(&mutex);
        current_state = ROAD_A_GREEN;
        printf("Traffic Light: Green for Road A, Red for Road B\n");
        pthread_cond_broadcast(&road_condition);
        pthread_mutex_unlock(&mutex);

        sleep(5);  // Green light duration

        // Road B green
        pthread_mutex_lock(&mutex);
        current_state = ROAD_B_GREEN;
        printf("Traffic Light: Green for Road B, Red for Road A\n");
        pthread_cond_broadcast(&road_condition);
        pthread_mutex_unlock(&mutex);

        sleep(5);  // Green light duration

        cycle_count++;
    }

    // Stop simulation
    stop_simulation = 1;
    pthread_cond_broadcast(&road_condition);

    return NULL;
}

int main() {
    pthread_t road_a, road_b, traffic_controller;

    // Create threads
    pthread_create(&road_a, NULL, road_a_thread, NULL);
    pthread_create(&road_b, NULL, road_b_thread, NULL);
    pthread_create(&traffic_controller, NULL, traffic_light_controller, NULL);

    // Wait for threads to complete
    pthread_join(road_a, NULL);
    pthread_join(road_b, NULL);
    pthread_join(traffic_controller, NULL);

    // Clean up
    pthread_mutex_destroy(&mutex);
    pthread_cond_destroy(&road_condition);

    return 0;
}

Overwriting l5t2.c


In [ ]:
!gcc l5t2.c

In [ ]:
!./a.out

Road A: Green light! Cars passing...
Road B: Red light. Waiting...
Traffic Light: Green for Road A, Red for Road B
Road B: Red light. Waiting...
Road A: Green light! Cars passing...
Road A: Green light! Cars passing...
Traffic Light: Green for Road B, Red for Road A
Road B: Green light! Cars passing...
^C


 Parallel Array Sum using OpenMP|

In [ ]:
%%writefile l5t3.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <omp.h>

#define ARRAY_SIZE 100
#define NUM_THREADS 4

void initialize_array(int arr[], int size) {
    // Seed the random number generator
    srand(time(NULL));

    // Fill array with random values between 1 and 1000
    for (int i = 0; i < size; i++) {
        arr[i] = rand() % 1000 + 1;
    }
}

int sequential_sum(int arr[], int size) {
    int sum = 0;
    for (int i = 0; i < size; i++) {
        sum += arr[i];
    }
    return sum;
}

int parallel_sum(int arr[], int size) {
    int sum = 0;

    // Set the number of threads
    omp_set_num_threads(NUM_THREADS);

    // Parallel reduction to calculate sum
    #pragma omp parallel for reduction(+:sum)
    for (int i = 0; i < size; i++) {
        sum += arr[i];
    }

    return sum;
}

void print_array(int arr[], int size) {
    printf("Array elements:\n");
    for (int i = 0; i < size; i++) {
        printf("%d ", arr[i]);
        if ((i + 1) % 10 == 0) {
            printf("\n");
        }
    }
    printf("\n");
}

int main() {
    int arr[ARRAY_SIZE];
    int sequential_result, parallel_result;
    double start_time, end_time;

    // Initialize the array with random values
    initialize_array(arr, ARRAY_SIZE);

    // Print array elements
    print_array(arr, ARRAY_SIZE);

    // Sequential Sum
    start_time = omp_get_wtime();
    sequential_result = sequential_sum(arr, ARRAY_SIZE);
    end_time = omp_get_wtime();
    printf("Sequential Sum: %d\n", sequential_result);
    printf("Sequential Time: %f seconds\n", end_time - start_time);

    // Parallel Sum
    start_time = omp_get_wtime();
    parallel_result = parallel_sum(arr, ARRAY_SIZE);
    end_time = omp_get_wtime();
    printf("Parallel Sum: %d\n", parallel_result);
    printf("Parallel Time: %f seconds\n", end_time - start_time);

    // Verify results
    if (sequential_result == parallel_result) {
        printf("Results match! Parallel computation successful.\n");
    } else {
        printf("Error: Results do not match!\n");
    }

    return 0;
}

Writing l5t3.c


In [ ]:
!gcc -fopenmp l5t3.c

In [ ]:
!./a.out

Array elements:
176 336 325 534 26 268 51 489 78 393 
611 91 572 877 595 342 788 306 277 65 
960 275 548 421 865 618 474 23 758 643 
831 934 330 155 467 356 774 869 196 852 
262 806 294 833 34 888 175 822 545 803 
238 505 429 138 277 646 107 750 668 865 
744 850 798 426 356 616 781 129 484 976 
332 745 133 978 578 167 865 104 340 762 
258 577 266 687 66 542 332 173 643 351 
37 739 200 186 164 907 801 944 387 284 

Sequential Sum: 48342
Sequential Time: 0.000004 seconds
Parallel Sum: 48342
Parallel Time: 0.000992 seconds
Results match! Parallel computation successful.


Using Sections for Text Analysis

In [ ]:
%%writefile l6t1.c

#include <stdio.h>
#include <string.h>
#include <ctype.h>
#include <omp.h>

// Function to count words in the text
int count_words(const char* text) {
    int word_count = 0;
    int in_word = 0;

    while (*text) {
        if (isspace(*text)) {
            in_word = 0;
        } else if (!in_word) {
            in_word = 1;
            word_count++;
        }
        text++;
    }

    return word_count;
}

// Function to count sentences in the text
int count_sentences(const char* text) {
    int sentence_count = 0;

    while (*text) {
        if (*text == '.' || *text == '!' || *text == '?') {
            sentence_count++;
        }
        text++;
    }

    return sentence_count;
}

// Function to count paragraphs in the text
int count_paragraphs(const char* text) {
    int paragraph_count = 1;
    int newline_count = 0;

    while (*text) {
        if (*text == '\n') {
            newline_count++;
            if (newline_count >= 2) {
                paragraph_count++;
                newline_count = 0;
            }
        } else if (!isspace(*text)) {
            newline_count = 0;
        }
        text++;
    }

    return paragraph_count;
}

int main() {
    // Sample text for analysis
    const char* text = "This is a sentence. This is another one! Here is a new paragraph.\n\nThis is a new line.";

    // Variables to store analysis results
    int word_count = 0;
    int sentence_count = 0;
    int paragraph_count = 0;

    // Use OpenMP sections to perform concurrent text analysis
    #pragma omp parallel sections
    {
        #pragma omp section
        {
            // Count words
            word_count = count_words(text);
            printf("Thread %d computing Word Count\n", omp_get_thread_num());
        }

        #pragma omp section
        {
            // Count sentences
            sentence_count = count_sentences(text);
            printf("Thread %d computing Sentence Count\n", omp_get_thread_num());
        }

        #pragma omp section
        {
            // Count paragraphs
            paragraph_count = count_paragraphs(text);
            printf("Thread %d computing Paragraph Count\n", omp_get_thread_num());
        }
    }

    // Print final analysis results
    printf("\nText Analysis Results:\n");
    printf("Text: \"%s\"\n", text);
    printf("Word Count: %d\n", word_count);
    printf("Sentence Count: %d\n", sentence_count);
    printf("Paragraph Count: %d\n", paragraph_count);

    return 0;
}

Writing l6t1.c


In [ ]:
!gcc -fopenmp l6t1.c

In [ ]:
!./a.out

Thread 1 computing Sentence Count
Thread 1 computing Paragraph Count
Thread 0 computing Word Count

Text Analysis Results:
Text: "This is a sentence. This is another one! Here is a new paragraph.

This is a new line."
Word Count: 18
Sentence Count: 4
Paragraph Count: 2


In [ ]:
%%writefile l6t1_1.c

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <ctype.h>

#define MAX_TEXT_LENGTH 1000

// Robust word counting function
int count_words(const char* text) {
    int word_count = 0;
    int in_word = 0;

    while (*text) {
        // Skip leading whitespace
        if (isspace(*text)) {
            in_word = 0;
        }
        // Check for start of a word
        else if (!in_word) {
            word_count++;
            in_word = 1;
        }
        text++;
    }

    return word_count;
}

// Robust sentence counting function
int count_sentences(const char* text) {
    int sentence_count = 0;
    int i = 0;

    while (text[i]) {
        // Check for sentence-ending punctuation
        if (text[i] == '.' || text[i] == '!' || text[i] == '?') {
            // Ensure it's a valid sentence end
            if (i > 0 &&
                (isalnum(text[i-1]) ||
                 (i > 1 && isalnum(text[i-2]) && ispunct(text[i-1])))) {
                sentence_count++;
            }
        }
        i++;
    }

    return sentence_count;
}

// Robust paragraph counting function
int count_paragraphs(const char* text) {
    int paragraph_count = 1;
    int newline_count = 0;
    int i = 0;

    while (text[i]) {
        if (text[i] == '\n') {
            newline_count++;

            // Two or more consecutive newlines indicate a new paragraph
            if (newline_count >= 2) {
                paragraph_count++;
                newline_count = 0;
            }
        } else if (!isspace(text[i])) {
            // Reset newline count when non-whitespace character is found
            newline_count = 0;
        }
        i++;
    }

    return paragraph_count;
}

int main(int argc, char** argv) {
    // Initialize MPI
    MPI_Init(&argc, &argv);

    // Get process rank and total number of processes
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Sample text
    char full_text[MAX_TEXT_LENGTH] =
        "This is a sentence. This is another one! Here is a new paragraph.\n\nThis is a new line.";

    // Ensure null-termination
    full_text[MAX_TEXT_LENGTH - 1] = '\0';

    // Results variables
    int word_count = 0;
    int sentence_count = 0;
    int paragraph_count = 0;

    // Perform analysis on root process
    if (rank == 0) {
        // Detailed debugging print
        printf("Full Text: [%s]\n", full_text);

        // Perform counts
        word_count = count_words(full_text);
        sentence_count = count_sentences(full_text);
        paragraph_count = count_paragraphs(full_text);
    }

    // Print results in root process
    if (rank == 0) {
        printf("Text Analysis Results:\n");
        printf("Text: \"%s\"\n", full_text);
        printf("Word Count: %d\n", word_count);
        printf("Sentence Count: %d\n", sentence_count);
        printf("Paragraph Count: %d\n", paragraph_count);
    }

    // Finalize MPI
    MPI_Finalize();

    return 0;
}

Overwriting l6t1_1.c


In [ ]:
!mpicc -o t1 l6t1_1.c

In [ ]:
!mpirun --allow-run-as-root t1

Full Text: [This is a sentence. This is another one! Here is a new paragraph.

This is a new line.]
Text Analysis Results:
Text: "This is a sentence. This is another one! Here is a new paragraph.

This is a new line."
Word Count: 18
Sentence Count: 4
Paragraph Count: 2


Calculations with OpenMP Implement a program that processes an array of integers to compute the sum, product, and maximum value concurrently.

In [ ]:
%%writefile l6t2.c

#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <limits.h>

#define ARRAY_SIZE 5
#define NUM_THREADS 4

// Function to compute sum using tasks
int compute_sum(int arr[], int start, int end) {
    int local_sum = 0;

    // Base case: single element
    if (start == end) {
        return arr[start];
    }

    // Recursive task division
    #pragma omp task shared(local_sum) firstprivate(start, end)
    {
        int mid = (start + end) / 2;
        int left_sum = compute_sum(arr, start, mid);
        int right_sum = compute_sum(arr, mid + 1, end);

        #pragma omp critical
        {
            local_sum = left_sum + right_sum;
        }
    }
    #pragma omp taskwait

    return local_sum;
}

// Function to compute product using tasks
long long compute_product(int arr[], int start, int end) {
    long long local_product = 1;

    // Base case: single element
    if (start == end) {
        return arr[start];
    }

    // Recursive task division
    #pragma omp task shared(local_product) firstprivate(start, end)
    {
        int mid = (start + end) / 2;
        long long left_product = compute_product(arr, start, mid);
        long long right_product = compute_product(arr, mid + 1, end);

        #pragma omp critical
        {
            local_product = left_product * right_product;
        }
    }
    #pragma omp taskwait

    return local_product;
}

// Function to find maximum value
int find_maximum(int arr[], int size) {
    int max_val = INT_MIN;

    #pragma omp parallel for reduction(max:max_val)
    for (int i = 0; i < size; i++) {
        if (arr[i] > max_val) {
            max_val = arr[i];
        }
    }

    return max_val;
}

int main() {
    // Initialize array
    int arr[ARRAY_SIZE] = {1, 2, 3, 4, 5};

    // Variables to store results
    int sum = 0;
    long long product = 1;
    int maximum = 0;

    // Set number of threads
    omp_set_num_threads(NUM_THREADS);

    // Print input array
    printf("Array: [");
    for (int i = 0; i < ARRAY_SIZE; i++) {
        printf("%d%s", arr[i], (i < ARRAY_SIZE - 1) ? ", " : "");
    }
    printf("]\n");

    // Use OpenMP sections to manage computations
    #pragma omp parallel
    {
        // Sum computation using tasks
        #pragma omp sections
        {
            #pragma omp section
            {
                #pragma omp task
                {
                    sum = compute_sum(arr, 0, ARRAY_SIZE - 1);
                }
            }

            // Product computation using tasks
            #pragma omp section
            {
                #pragma omp task
                {
                    product = compute_product(arr, 0, ARRAY_SIZE - 1);
                }
            }

            // Maximum value computation
            #pragma omp section
            {
                maximum = find_maximum(arr, ARRAY_SIZE);
            }
        }

        // Wait for all tasks to complete
        #pragma omp taskwait
    }

    // Print results
    printf("Sum: %d\n", sum);
    printf("Product: %lld\n", product);
    printf("Maximum: %d\n", maximum);

    return 0;
}

Writing l6t2.c


In [ ]:
!gcc -fopenmp l6t2.c
!./a.out

Array: [1, 2, 3, 4, 5]
Sum: 15
Product: 120
Maximum: 5


Fibonacci Series Calculation

In [ ]:
%%writefile l6t3.c

#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

// Recursive Fibonacci function with OpenMP tasks
int fibonacci(int n) {
    // Base cases
    if (n <= 1) {
        return n;
    }

    int left, right;

    // Create tasks for left and right branches
    #pragma omp task shared(left) firstprivate(n)
    {
        left = fibonacci(n - 1);
    }

    #pragma omp task shared(right) firstprivate(n)
    {
        right = fibonacci(n - 2);
    }

    // Wait for both tasks to complete
    #pragma omp taskwait

    // Return sum of left and right branches
    return left + right;
}

// Function to generate and print Fibonacci series
void generate_fibonacci_series(int n) {
    // Validate input
    if (n <= 0) {
        printf("Please enter a positive integer.\n");
        return;
    }

    // Array to store Fibonacci numbers
    int* series = (int*)malloc((n + 1) * sizeof(int));

    // Set number of threads
    omp_set_num_threads(4);

    // Parallel region for Fibonacci calculation
    #pragma omp parallel
    {
        #pragma omp single
        {
            // Generate Fibonacci series
            for (int i = 0; i < n; i++) {
                #pragma omp task shared(series) firstprivate(i)
                {
                    series[i] = fibonacci(i);
                }
            }

            // Wait for all tasks to complete
            #pragma omp taskwait
        }
    }

    // Print Fibonacci series
    printf("Fibonacci series up to %d:\n", n);
    for (int i = 0; i < n; i++) {
        printf("%d%s", series[i], (i < n - 1) ? ", " : "\n");
    }

    // Free allocated memory
    free(series);
}

// Input validation function
int get_positive_integer() {
    int n;

    while (1) {
        printf("Enter a positive integer: ");

        // Check if input is valid integer
        if (scanf("%d", &n) != 1) {
            // Clear input buffer
            while (getchar() != '\n');
            printf("Invalid input. Please enter a positive integer.\n");
            continue;
        }

        // Check if input is positive
        if (n > 0) {
            return n;
        }

        printf("Please enter a positive integer.\n");
    }
}

int main() {
    // Get user input
    int n = get_positive_integer();

    // Generate and print Fibonacci series
    generate_fibonacci_series(n);

    return 0;
}

Writing l6t3.c


In [ ]:
!gcc -fopenmp l6t3.c

!./a.out

Enter a positive integer: 10
Fibonacci series up to 10:
0, 1, 1, 2, 3, 5, 8, 13, 21, 34


Linked List Traversal

In [ ]:
%%writefile l6h.c



#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

// Linked List Node Structure
typedef struct Node {
    int data;
    struct Node* next;
} Node;

// Function to create a new node
Node* create_node(int value) {
    Node* new_node = (Node*)malloc(sizeof(Node));
    if (new_node == NULL) {
        printf("Memory allocation failed!\n");
        exit(1);
    }
    new_node->data = value;
    new_node->next = NULL;
    return new_node;
}

// Function to insert a node at the end of the list
void insert_node(Node** head, int value) {
    Node* new_node = create_node(value);

    if (*head == NULL) {
        *head = new_node;
        return;
    }

    Node* current = *head;
    while (current->next != NULL) {
        current = current->next;
    }
    current->next = new_node;
}

// Function to print a single node (task function)
void print_node(Node* node, int* order) {
    #pragma omp critical
    {
        printf("Node value: %d (Order: %d)\n", node->data, (*order)++);
    }
}

// Parallel traversal function using OpenMP tasks
void parallel_traverse(Node* head) {
    // Order tracking for consistent output
    int order = 1;

    // Set number of threads
    omp_set_num_threads(4);

    // Parallel region
    #pragma omp parallel
    {
        #pragma omp single
        {
            Node* current = head;

            // Traverse the list
            while (current != NULL) {
                // Create a task for each node
                #pragma omp task firstprivate(current)
                {
                    print_node(current, &order);
                }

                current = current->next;
            }

            // Wait for all tasks to complete
            #pragma omp taskwait
        }
    }
}

// Function to free linked list memory
void free_list(Node* head) {
    Node* current = head;
    Node* next;

    while (current != NULL) {
        next = current->next;
        free(current);
        current = next;
    }
}

// Function to create a sample linked list
Node* create_sample_list() {
    Node* head = NULL;

    // Insert sample values
    insert_node(&head, 10);
    insert_node(&head, 20);
    insert_node(&head, 30);
    insert_node(&head, 40);
    insert_node(&head, 50);

    return head;
}

// Main function
int main() {
    // Create sample linked list
    Node* head = create_sample_list();

    // Print header
    printf("Parallel Linked List Traversal:\n");

    // Perform parallel traversal
    parallel_traverse(head);

    // Free allocated memory
    free_list(head);

    return 0;
}

Writing l6h.c


In [ ]:
!gcc -fopenmp l6h.c

!./a.out

Parallel Linked List Traversal:
Node value: 50 (Order: 1)
Node value: 40 (Order: 2)
Node value: 30 (Order: 3)
Node value: 20 (Order: 4)
Node value: 10 (Order: 5)


Transpose of the matrix

In [ ]:
%%writefile l6.cu

#include <iostream>
#include <cuda_runtime.h>
#include <cstdlib>
#include <cassert>

#define TILE_SIZE 32 // Tile size for shared memory optimization

// CUDA kernel for transposing a square matrix
__global__ void transposeMatrix(float *input, float *output, int n) {
    // Shared memory with padding to avoid bank conflicts
    __shared__ float tile[TILE_SIZE][TILE_SIZE + 1];

    // Calculate global coordinates
    int x = blockIdx.x * TILE_SIZE + threadIdx.x;
    int y = blockIdx.y * TILE_SIZE + threadIdx.y;

    // Load input matrix to shared memory
    if (x < n && y < n) {
        tile[threadIdx.y][threadIdx.x] = input[y * n + x];
    }

    // Synchronize threads to ensure all data is loaded
    __syncthreads();

    // Recalculate coordinates for transposed output
    x = blockIdx.y * TILE_SIZE + threadIdx.x;
    y = blockIdx.x * TILE_SIZE + threadIdx.y;

    // Write transposed data from shared memory
    if (x < n && y < n) {
        output[y * n + x] = tile[threadIdx.x][threadIdx.y];
    }
}

// CPU function for matrix transposition
void transposeMatrixCPU(float *input, float *output, int n) {
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            output[j * n + i] = input[i * n + j];
        }
    }
}

// Initialize matrix with random values within a range
void initializeMatrix(float *matrix, int n, int minValue, int maxValue) {
    for (int i = 0; i < n * n; ++i) {
        matrix[i] = minValue + rand() % (maxValue - minValue + 1);
    }
}

// Print a square matrix to the console
void printMatrix(float *matrix, int n) {
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            std::cout << matrix[i * n + j] << " ";
        }
        std::cout << std::endl;
    }
}

// Main function to demonstrate matrix transposition
int main() {
    // Matrix size
    const int n = 1024;

    // Allocate host memory
    float *h_input = new float[n * n];
    float *h_output_cuda = new float[n * n];
    float *h_output_cpu = new float[n * n];

    // Initialize input matrix with random values
    srand(time(NULL));
    initializeMatrix(h_input, n, 1, 100);

    // Allocate device memory
    float *d_input, *d_output;
    cudaMalloc(&d_input, n * n * sizeof(float));
    cudaMalloc(&d_output, n * n * sizeof(float));

    // Copy input matrix to device
    cudaMemcpy(d_input, h_input, n * n * sizeof(float), cudaMemcpyHostToDevice);

    // Configure grid and block dimensions
    dim3 blockDim(TILE_SIZE, TILE_SIZE);
    dim3 gridDim(
        (n + blockDim.x - 1) / blockDim.x,
        (n + blockDim.y - 1) / blockDim.y
    );

    // Launch CUDA kernel for matrix transposition
    transposeMatrix<<<gridDim, blockDim>>>(d_input, d_output, n);

    // Copy transposed matrix back to host
    cudaMemcpy(h_output_cuda, d_output, n * n * sizeof(float), cudaMemcpyDeviceToHost);

    // Compute CPU reference transposition
    transposeMatrixCPU(h_input, h_output_cpu, n);

    // Verify results
    bool success = true;
    for (int i = 0; i < n * n; ++i) {
        if (std::abs(h_output_cuda[i] - h_output_cpu[i]) > 1e-5) {
            success = false;
            break;
        }
    }

    // Print verification result
    std::cout << "Matrix Transposition "
              << (success ? "Successful" : "Failed")
              << std::endl;

    // Free host memory
    delete[] h_input;
    delete[] h_output_cuda;
    delete[] h_output_cpu;

    // Free device memory
    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}

Overwriting l6.cu


In [ ]:
!nvcc -o l l6.cu
!./l

Matrix Transposition Successful


In [ ]:
%%writefile l81.cu

#include <cuda_runtime.h>
#include <iostream>
#include <iomanip>

// Kernel for adding a scalar to matrix
__global__ void addScalarKernel(float *input, float *output, float scalar, int rows, int cols) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < rows && col < cols) {
        int index = row * cols + col;
        output[index] = input[index] + scalar;
    }
}

// Kernel for multiplying matrix by a scalar
__global__ void multiplyScalarKernel(float *input, float *output, float scalar, int rows, int cols) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < rows && col < cols) {
        int index = row * cols + col;
        output[index] = input[index] * scalar;
    }
}

// Kernel for squaring matrix elements
__global__ void squareMatrixKernel(float *input, float *output, int rows, int cols) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < rows && col < cols) {
        int index = row * cols + col;
        output[index] = input[index] * input[index];
    }
}

// Function to print matrix
void printMatrix(float *matrix, int rows, int cols) {
    std::cout << "[";
    for (int i = 0; i < rows; i++) {
        if (i > 0) std::cout << " ";
        std::cout << "[";
        for (int j = 0; j < cols; j++) {
            std::cout << matrix[i * cols + j];
            if (j < cols - 1) std::cout << ", ";
        }
        std::cout << "]";
        if (i < rows - 1) std::cout << "," << std::endl;
    }
    std::cout << "]" << std::endl;
}

int main() {
    // Matrix dimensions
    const int rows = 3;
    const int cols = 3;
    const int size = rows * cols * sizeof(float);

    // Host matrices
    float h_input[rows * cols] = {
        1, 2, 3,
        4, 5, 6,
        7, 8, 9
    };
    float h_output[rows * cols];

    // Device matrices
    float *d_input, *d_output;

    // Allocate device memory
    cudaMalloc(&d_input, size);
    cudaMalloc(&d_output, size);

    // Copy input matrix to device
    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    dim3 blockDim(3, 3);
    dim3 gridDim(1, 1);

    // Print input matrix
    std::cout << "Input Matrix:" << std::endl;
    printMatrix(h_input, rows, cols);
    std::cout << std::endl;

    // 1. Add Scalar Operation
    float addScalar = 2.0f;
    addScalarKernel<<<gridDim, blockDim>>>(d_input, d_output, addScalar, rows, cols);
    cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost);

    std::cout << "Added Scalar:" << std::endl;
    printMatrix(h_output, rows, cols);
    std::cout << std::endl;

    // 2. Multiply Scalar Operation
    float multiplyScalar = 3.0f;
    multiplyScalarKernel<<<gridDim, blockDim>>>(d_input, d_output, multiplyScalar, rows, cols);
    cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost);

    std::cout << "Multiplied Scalar:" << std::endl;
    printMatrix(h_output, rows, cols);
    std::cout << std::endl;

    // 3. Square Operation
    squareMatrixKernel<<<gridDim, blockDim>>>(d_input, d_output, rows, cols);
    cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost);

    std::cout << "Squared:" << std::endl;
    printMatrix(h_output, rows, cols);

    // Free device memory
    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}

Overwriting l81.cu


In [ ]:
!nvcc l81.cu -o matrix_ops

# Run the executable
!./matrix_ops

Input Matrix:
[[1, 2, 3],
 [4, 5, 6],
 [7, 8, 9]]

Added Scalar:
[[3, 4, 5],
 [6, 7, 8],
 [9, 10, 11]]

Multiplied Scalar:
[[3, 6, 9],
 [12, 15, 18],
 [21, 24, 27]]

Squared:
[[1, 4, 9],
 [16, 25, 36],
 [49, 64, 81]]


In [ ]:
%%writefile l91.cpp

#include <mpi.h>
#include <iostream>

int main(int argc, char** argv) {
    // Initialize the MPI environment
    MPI_Init(&argc, &argv);

    // Get the number of processes
    int total_processes;
    MPI_Comm_size(MPI_COMM_WORLD, &total_processes);

    // Get the rank of the process
    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    // Print a hello world message
    std::cout << "Hello from process " << rank
              << " of " << total_processes << std::endl;

    // Finalize the MPI environment
    MPI_Finalize();

    return 0;
}

Writing l91.cpp


In [ ]:
!mpicxx -o hello_world_mpi l91.cpp
!mpirun --allow-run-as-root  --oversubscribe -np 4 ./hello_world_mpi

Hello from process 2 of 4
Hello from process 0 of 4
Hello from process 3 of 4
Hello from process 1 of 4


Pingpong

In [ ]:
# Comprehensive MPI Message Passing Setup

# Install OpenMPI
# !apt-get update && apt-get install -y openmpi-bin libopenmpi-dev

# Create MPI source file
%%writefile message_passing.cpp
#include <mpi.h>
#include <iostream>
#include <stdexcept>

int main(int argc, char** argv) {
    try {
        // Initialize MPI with error checking
        int mpi_init_result = MPI_Init(&argc, &argv);
        if (mpi_init_result != MPI_SUCCESS) {
            throw std::runtime_error("MPI Initialization failed");
        }

        // Get process information
        int rank, total_processes;
        MPI_Comm_rank(MPI_COMM_WORLD, &rank);
        MPI_Comm_size(MPI_COMM_WORLD, &total_processes);

        // Validate process count
        if (total_processes != 2) {
            throw std::runtime_error("This program requires exactly 2 processes");
        }

        int number = 10;
        const int exchange_count = 5;

        // Process 0 initiates communication
        if (rank == 0) {
            for (int i = 0; i < exchange_count; ++i) {
                std::cout << "Process 0 sent number " << number
                          << " to Process 1" << std::endl;
                MPI_Send(&number, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);

                MPI_Recv(&number, 1, MPI_INT, 1, 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
                std::cout << "Process 0 received number " << number << std::endl;
            }
        }
        // Process 1 receives and sends back
        else if (rank == 1) {
            for (int i = 0; i < exchange_count; ++i) {
                MPI_Recv(&number, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
                std::cout << "Process 1 received number " << number
                          << ", incremented to " << number + 1
                          << ", and sent it back" << std::endl;

                ++number;
                MPI_Send(&number, 1, MPI_INT, 0, 1, MPI_COMM_WORLD);
            }
        }

        // Finalize MPI
        MPI_Finalize();
        return 0;
    }
    catch (const std::exception& e) {
        std::cerr << "Error: " << e.what() << std::endl;
        MPI_Finalize();
        return 1;
    }
}



Overwriting message_passing.cpp


In [ ]:
# Compile MPI program
!mpicxx message_passing.cpp -o message_passing

# Execution strategies
execution_modes = [
    "mpirun --allow-run-as-root --oversubscribe -np 2",
    "mpirun -np 2"
]

# Try different execution configurations
for mode in execution_modes:
    print(f"\nExecuting with: {mode}")
    try:
        !{mode} ./message_passing
    except Exception as e:
        print(f"Execution failed: {e}")


Executing with: mpirun --allow-run-as-root --oversubscribe -np 2
Process 0 sent number 10 to Process 1
Process 1 received number 10, incremented to 11, and sent it back
Process 0 received number 11
Process 0 sent number 11 to Process 1
Process 1 received number 11, incremented to 12, and sent it back
Process 0 received number 12
Process 0 sent number 12 to Process 1
Process 1 received number 12, incremented to 13, and sent it back
Process 0 received number 13
Process 0 sent number 13 to Process 1
Process 1 received number 13, incremented to 14, and sent it back
Process 0 received number 14
Process 0 sent number 14 to Process 1
Process 1 received number 14, incremented to 15, and sent it back
Process 0 received number 15

Executing with: mpirun -np 2
--------------------------------------------------------------------------
mpirun has detected an attempt to run as root.

Running as root is *strongly* discouraged as any mistake (e.g., in
defining TMPDIR) or bug can result in catastrophi

In [ ]:
# MPI Reverse Number Passing Setup and Execution

# Step 1: Install OpenMPI
# !apt-get update && apt-get install -y openmpi-bin libopenmpi-dev

# Step 2: Create the MPI Reverse Number Passing source file
%%writefile reverse_number_passing.cpp
#include <mpi.h>
#include <iostream>
#include <stdexcept>

int main(int argc, char** argv) {
    try {
        // Initialize the MPI environment
        MPI_Init(&argc, &argv);

        // Get the number of processes
        int total_processes;
        MPI_Comm_size(MPI_COMM_WORLD, &total_processes);

        // Get the rank of the current process
        int rank;
        MPI_Comm_rank(MPI_COMM_WORLD, &rank);

        // Initial number (set by the last process)
        int number;

        // Last process initializes the number
        if (rank == total_processes - 1) {
            number = 5;  // Starting number
            std::cout << "Process " << rank << ": Starting number is "
                      << number << ", multiplied by " << rank
                      << ", sending " << number * rank << std::endl;

            // Multiply by rank and send to previous process
            number *= rank;
            MPI_Send(&number, 1, MPI_INT, rank - 1, 0, MPI_COMM_WORLD);
        }
        // Intermediate processes
        else if (rank > 0 && rank < total_processes - 1) {
            // Receive from the next process
            MPI_Recv(&number, 1, MPI_INT, rank + 1, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

            std::cout << "Process " << rank << ": Received " << number
                      << ", multiplied by " << rank
                      << ", sending " << number * rank << std::endl;

            // Multiply by rank and send to previous process
            number *= rank;
            MPI_Send(&number, 1, MPI_INT, rank - 1, 0, MPI_COMM_WORLD);
        }
        // First process (Process 0)
        else if (rank == 0) {
            // Receive final number from Process 1
            MPI_Recv(&number, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

            std::cout << "Process 0: Received " << number << std::endl;
        }

        // Finalize the MPI environment
        MPI_Finalize();
        return 0;
    }
    catch (const std::exception& e) {
        std::cerr << "Error: " << e.what() << std::endl;
        MPI_Finalize();
        return 1;
    }
}


Overwriting reverse_number_passing.cpp


In [ ]:

# Step 3: Compile the MPI program
!mpicxx reverse_number_passing.cpp -o reverse_number_passing

# Step 4: Run the MPI program with 4 processes
!mpirun --allow-run-as-root --oversubscribe -np 4 ./reverse_number_passing

Process 3: Starting number is 5, multiplied by 3, sending 15
Process 2: Received 15, multiplied by 2, sending 30
Process 1: Received 30, multiplied by 1, sending 30
Process 0: Received 30


In [73]:
%%writefile broadcast.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank;
    int data = 0;

    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    if (rank == 0) {
        data = 100; // root process sets the data
    }

    MPI_Bcast(&data, 1, MPI_INT, 0, MPI_COMM_WORLD);

    printf("Process %d received data %d\n", rank, data);

    MPI_Finalize();
    return 0;
}

Writing broadcast.c


In [74]:
!mpicc broadcast.c
!mpirun --allow-run-as-root --oversubscribe -np 4 ./a.out

Process 0 received data 100
Process 2 received data 100
Process 3 received data 100
Process 1 received data 100


In [75]:
%%writefile gather.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    int send_data, recv_data[4];

    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    send_data = rank + 1; // each process has a unique value

    MPI_Gather(&send_data, 1, MPI_INT, recv_data, 1, MPI_INT, 0, MPI_COMM_WORLD);

    if (rank == 0) {
        printf("Root process gathered data: ");
        for (int i = 0; i < size; i++) {
            printf("%d ", recv_data[i]);
        }
        printf("\n");
    }

    MPI_Finalize();
    return 0;
}

Writing gather.c


In [76]:
!mpicc gather.c

!mpirun --allow-run-as-root --oversubscribe -np 4 ./a.out

Root process gathered data: 1 2 3 4 


In [77]:
%%writefile p2p.c

#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    if (rank == 0) {
        int data = 42;
        MPI_Send(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
        printf("Process 0 sent data %d\n", data);
    } else if (rank == 1) {
        int data;
        MPI_Recv(&data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Process 1 received data %d\n", data);
    }

    MPI_Finalize();
    return 0;
}

Writing p2p.c


In [78]:
!mpicc p2p.c
!mpirun --allow-run-as-root --oversubscribe -np 4 ./a.out

Process 0 sent data 42
Process 1 received data 42


In [79]:
%%writefile reduce.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    int send_data, result;

    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    send_data = rank + 1; // each process contributes its rank + 1

    MPI_Reduce(&send_data, &result, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

    if (rank == 0) {
        printf("Global sum of ranks is %d\n", result);
    }

    MPI_Finalize();
    return 0;
}

Writing reduce.c


In [80]:
!mpicc reduce.c
!mpirun --allow-run-as-root --oversubscribe -np 4 ./a.out

Global sum of ranks is 10


In [81]:
%%writefile scatter.c

#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int data[4] = {10, 20, 30, 40}; // root's data
    int recv_data;

    MPI_Scatter(data, 1, MPI_INT, &recv_data, 1, MPI_INT, 0, MPI_COMM_WORLD);

    printf("Process %d received data %d\n", rank, recv_data);

    MPI_Finalize();
    return 0;
}

Writing scatter.c


In [82]:
!mpicc scatter.c
!mpirun --allow-run-as-root --oversubscribe -np 4 ./a.out

Process 0 received data 10
Process 2 received data 30
Process 3 received data 40
Process 1 received data 20


In [83]:
%%writefile task1.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank;
    int data = 0;

    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    if (rank == 0) {
        data = 100; // root process sets the data
    }

    MPI_Bcast(&data, 1, MPI_INT, 0, MPI_COMM_WORLD);

    printf("Process %d received data %d\n", rank, data);

    MPI_Finalize();
    return 0;
}

Writing task1.c


In [84]:
!mpicc -o task1 task1.c
!mpirun --allow-run-as-root --oversubscribe -np 4 ./task1

Process 0 received data 100
Process 2 received data 100
Process 1 received data 100
Process 3 received data 100


In [85]:
%%writefile task2.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int send_data[4];
    int recv_data;

    if (rank == 0) {
        // Initialize the array in process 0
        for (int i = 0; i < 4; i++) {
            send_data[i] = i + 1;
            printf("%d ", send_data[i]);
        }
    }

    MPI_Scatter(send_data, 1, MPI_INT, &recv_data, 1, MPI_INT, 0, MPI_COMM_WORLD);


    // Double the received value
    recv_data *= 2;

    int gathered_data[4];
    MPI_Gather(&recv_data, 1, MPI_INT, gathered_data, 1, MPI_INT, 0, MPI_COMM_WORLD);

    if (rank == 0) {
        printf("\nModified Array: ");
        for (int i = 0; i < size; i++) {
            printf("%d ", gathered_data[i]);
        }
        printf("\n");
    }

    MPI_Finalize();
    return 0;
}

Writing task2.c


In [86]:
!mpicc -o task2 task2.c

!mpirun --allow-run-as-root --oversubscribe -np 4 ./task2

1 2 3 4 
Modified Array: 2 4 6 8 


In [87]:
%%writefile task3.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int local_value = rank + 1;
    int global_sum = 0;

    MPI_Reduce(&local_value, &global_sum, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

    if (rank == 0) {
        printf("Global sum is %d\n", global_sum);
    }

    MPI_Finalize();
    return 0;
}

Writing task3.c


In [88]:
!mpicc -o task3 task3.c
!mpirun --allow-run-as-root --oversubscribe -np 4 ./task3

Global sum is 10


In [93]:
%%writefile quicksort_openmp.c

#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <time.h>

// Function to swap two elements
void swap(int* a, int* b) {
    int temp = *a;
    *a = *b;
    *b = temp;
}

// Partition function for Quicksort
int partition(int* arr, int low, int high) {
    // Choose the rightmost element as pivot
    int pivot = arr[high];

    // Index of smaller element
    int i = (low - 1);

    // Traverse through all elements
    for (int j = low; j <= high - 1; j++) {
        // If current element is smaller than or equal to pivot
        if (arr[j] <= pivot) {
            // Increment index of smaller element
            i++;
            swap(&arr[i], &arr[j]);
        }
    }

    // Place pivot in correct position
    swap(&arr[i + 1], &arr[high]);
    return (i + 1);
}

// Sequential Quicksort implementation
void quicksortSequential(int* arr, int low, int high) {
    if (low < high) {
        // Find partition index
        int pi = partition(arr, low, high);

        // Recursively sort elements before and after partition
        quicksortSequential(arr, low, pi - 1);
        quicksortSequential(arr, pi + 1, high);
    }
}

// Parallel Quicksort implementation using OpenMP
void quicksortParallel(int* arr, int low, int high, int depth) {
    if (low < high) {
        // Find partition index
        int pi = partition(arr, low, high);

        // Check if we should use parallel or sequential sorting
        if (depth > 0) {
            #pragma omp parallel sections
            {
                #pragma omp section
                {
                    quicksortParallel(arr, low, pi - 1, depth - 1);
                }

                #pragma omp section
                {
                    quicksortParallel(arr, pi + 1, high, depth - 1);
                }
            }
        }
        else {
            // Fall back to sequential sorting when depth is reached
            quicksortSequential(arr, low, pi - 1);
            quicksortSequential(arr, pi + 1, high);
        }
    }
}

// Function to print array
void printArray(int* arr, int size, const char* message) {
    printf("%s: ", message);
    for (int i = 0; i < size; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");
}

// Function to generate random array
void generateRandomArray(int* arr, int size) {
    srand(time(NULL));
    for (int i = 0; i < size; i++) {
        arr[i] = rand() % 1000;  // Random numbers between 0 and 999
    }
}

// Function to verify if array is sorted
int isSorted(int* arr, int size) {
    for (int i = 1; i < size; i++) {
        if (arr[i] < arr[i-1]) {
            return 0;  // Not sorted
        }
    }
    return 1;  // Sorted
}

int main() {
    // Array size
    const int SIZE = 10;

    // Allocate memory for arrays
    int* arr_sequential = (int*)malloc(SIZE * sizeof(int));
    int* arr_parallel = (int*)malloc(SIZE * sizeof(int));

    // Generate random arrays
    generateRandomArray(arr_sequential, SIZE);

    // Print original array
    printArray(arr_sequential, SIZE, "Original Array");

    // Copy array for parallel sorting
    for (int i = 0; i < SIZE; i++) {
        arr_parallel[i] = arr_sequential[i];
    }

    // Set number of threads
    omp_set_num_threads(4);

    // Measure sequential sorting time
    printf("\n--- Sequential Sorting ---\n");
    double start_sequential = omp_get_wtime();
    quicksortSequential(arr_sequential, 0, SIZE - 1);
    double end_sequential = omp_get_wtime();

    // Print sequentially sorted array
    printArray(arr_sequential, SIZE, "Sequential Sorted Array");

    // Check if sequential sort worked
    if (!isSorted(arr_sequential, SIZE)) {
        printf("Sequential sort failed!\n");
        return 1;
    }

    // Measure parallel sorting time
    printf("\n--- Parallel Sorting ---\n");
    double start_parallel = omp_get_wtime();
    quicksortParallel(arr_parallel, 0, SIZE - 1, 3);  // Depth of 3
    double end_parallel = omp_get_wtime();

    // Print parallelly sorted array
    printArray(arr_parallel, SIZE, "Parallel Sorted Array");

    // Check if parallel sort worked
    if (!isSorted(arr_parallel, SIZE)) {
        printf("Parallel sort failed!\n");
        return 1;
    }

    // Print performance results
    printf("\n--- Performance Metrics ---\n");
    printf("Sequential Sort Time: %f seconds\n", end_sequential - start_sequential);
    printf("Parallel Sort Time: %f seconds\n", end_parallel - start_parallel);

    // Verify both arrays are identical
    int is_correct = 1;
    for (int i = 0; i < SIZE; i++) {
        if (arr_sequential[i] != arr_parallel[i]) {
            is_correct = 0;
            break;
        }
    }

    printf("Sort Correctness: %s\n", is_correct ? "Verified" : "Failed");

    // Free allocated memory
    free(arr_sequential);
    free(arr_parallel);

    return 0;
}

Overwriting quicksort_openmp.c


In [94]:
# Compile with OpenMP support
!gcc -fopenmp quicksort_openmp.c -o quicksort_openmp

# Run the executable
!./quicksort_openmp

Original Array: 227 358 705 567 399 853 307 566 354 348 

--- Sequential Sorting ---
Sequential Sorted Array: 227 307 348 354 358 399 566 567 705 853 

--- Parallel Sorting ---
Parallel Sorted Array: 227 307 348 354 358 399 566 567 705 853 

--- Performance Metrics ---
Sequential Sort Time: 0.000001 seconds
Parallel Sort Time: 0.000192 seconds
Sort Correctness: Verified


In [97]:
# Write the MPI source code
%%writefile prime_mpi.cpp

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

// Function to check if a number is prime
int isPrime(int num) {
    if (num <= 1) return 0;
    if (num <= 3) return 1;

    // Check for divisibility
    for (int i = 2; i <= sqrt(num); i++) {
        if (num % i == 0) return 0;
    }
    return 1;
}

int main(int argc, char** argv) {
    // Initialize MPI environment
    MPI_Init(&argc, &argv);

    // Get total number of processes and current process rank
    int total_processes, rank;
    MPI_Comm_size(MPI_COMM_WORLD, &total_processes);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    // Define range of numbers to check
    int start_range = 1;
    int end_range = 10;

    // Calculate range for each process
    int range_per_process = (end_range - start_range + 1) / total_processes;
    int remainder = (end_range - start_range + 1) % total_processes;

    // Allocate arrays for scatter and gather
    int* send_counts = NULL;
    int* displacements = NULL;
    int* local_numbers = NULL;
    int* local_primes = NULL;
    int* global_primes = NULL;

    if (rank == 0) {
        // Allocate memory for send counts and displacements
        send_counts = (int*)malloc(total_processes * sizeof(int));
        displacements = (int*)malloc(total_processes * sizeof(int));

        // Calculate send counts and displacements
        for (int i = 0; i < total_processes; i++) {
            send_counts[i] = range_per_process + (i < remainder ? 1 : 0);

            // Calculate displacement
            displacements[i] = (i == 0) ? start_range :
                (displacements[i-1] + send_counts[i-1]);
        }

        // Allocate global numbers array
        local_numbers = (int*)malloc(end_range * sizeof(int));
        for (int i = 0; i < end_range; i++) {
            local_numbers[i] = start_range + i;
        }
    }

    // Broadcast send counts to all processes
    int local_count;
    MPI_Scatter(send_counts, 1, MPI_INT, &local_count, 1, MPI_INT, 0, MPI_COMM_WORLD);

    // Allocate local numbers array for each process
    int* process_numbers = (int*)malloc(local_count * sizeof(int));
    int* process_primes = (int*)malloc(local_count * sizeof(int));

    // Scatter numbers to all processes
    MPI_Scatterv(local_numbers, send_counts, displacements, MPI_INT,
                 process_numbers, local_count, MPI_INT, 0, MPI_COMM_WORLD);

    // Find local primes
    int local_prime_count = 0;
    for (int i = 0; i < local_count; i++) {
        if (isPrime(process_numbers[i])) {
            process_primes[local_prime_count++] = process_numbers[i];
        }
    }

    // Gather prime counts from all processes
    int* prime_counts = NULL;
    if (rank == 0) {
        prime_counts = (int*)malloc(total_processes * sizeof(int));
    }

    // Gather local prime counts
    MPI_Gather(&local_prime_count, 1, MPI_INT,
               prime_counts, 1, MPI_INT, 0, MPI_COMM_WORLD);

    // Gather prime numbers
    int total_prime_count = 0;
    int* recv_displacements = NULL;
    int* recv_counts = NULL;

    if (rank == 0) {
        // Calculate total prime count and displacements
        recv_displacements = (int*)malloc(total_processes * sizeof(int));
        recv_counts = (int*)malloc(total_processes * sizeof(int));

        total_prime_count = 0;
        for (int i = 0; i < total_processes; i++) {
            recv_counts[i] = prime_counts[i];
            recv_displacements[i] = (i == 0) ? 0 : (recv_displacements[i-1] + recv_counts[i-1]);
            total_prime_count += prime_counts[i];
        }

        // Allocate global primes array
        global_primes = (int*)malloc(total_prime_count * sizeof(int));
    }

    // Gatherv prime numbers
    MPI_Gatherv(process_primes, local_prime_count, MPI_INT,
                global_primes, recv_counts, recv_displacements,
                MPI_INT, 0, MPI_COMM_WORLD);

    // Print results on root process
    if (rank == 0) {
        printf("Total Prime Numbers Found: %d\n", total_prime_count);
        printf("Prime Numbers: ");
        for (int i = 0; i < total_prime_count; i++) {
            printf("%d ", global_primes[i]);
        }
        printf("\n");

        // Free allocated memory
        free(send_counts);
        free(displacements);
        free(local_numbers);
        free(prime_counts);
        free(recv_displacements);
        free(recv_counts);
        free(global_primes);
    }

    // Free local process memory
    free(process_numbers);
    free(process_primes);

    // Finalize MPI environment
    MPI_Finalize();

    return 0;
}


Overwriting prime_mpi.cpp


In [98]:

# Compile the MPI program
!mpicxx prime_mpi.cpp -o prime_mpi

# Run the MPI program
!mpirun --allow-run-as-root --oversubscribe -np 4 ./prime_mpi

Total Prime Numbers Found: 4
Prime Numbers: 2 3 5 7 
